In [ ]:
pip install fake_useragent


In [ ]:
import fake_useragent
import requests
import json
import pandas as pd
import time
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
nltk.download('stopwords')
from nltk import ngrams
from collections import Counter
import string
from nltk.probability import FreqDist
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from wordcloud import WordCloud
import seaborn as sns

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Получение данных с сайт HeadHunter

In [ ]:
def headers():
    user_ag = fake_useragent.UserAgent().random
    headers = {'user-agent': user_ag}
    return headers

In [ ]:
def get_vacancies(text, page, country_code):

  params = {
        'text': text,
        'area': country_code,
        'per_page': 100,
        'page': page}


  url = 'https://api.hh.ru/vacancies'
  req = requests.get(url, params, headers=headers())
  data = req.content.decode()
  req.close()
  return data

In [ ]:
text = 'Data Analyst'
russia_country_code = 113

In [ ]:
# test_text = 'Junior data analyst'

In [ ]:
# page = json.loads(get_vacancies(test_text, 0, russia_country_code))
# page['pages']

1

In [ ]:
page = json.loads(get_vacancies(text, 0, russia_country_code))

In [ ]:
pages = page['pages']

In [ ]:
pages_info = []
for page in range(0, pages):
 pages_info.append(json.loads(get_vacancies(text, page, russia_country_code)))


In [ ]:
vacancies_info = []
for item in pages_info:
  for elements in item['items']:
    req = requests.get(elements['url'], headers=headers())
    vacancies_info.append(json.loads(req.content.decode()))
    time.sleep(0.25)
    req.close()

# Созадание датафрейма

In [ ]:
df_test = pd.DataFrame(columns=['vacancy_name', 'city','lat', 'lng', 'company_name','salary_from', 'salary_to',
                                'experience', 'description', 'skills'])

In [ ]:
def extract_data(data, first, second):
  first_data = data.get(first)
  if not first_data:
    return None
  else:
    second_data = first_data.get(second)
    return second_data

In [ ]:
for vacancy in vacancies_info:
    df_test.loc[len(df_test.index)] = {'vacancy_name' : vacancy.get('name'),
                                       'city' : extract_data(vacancy, 'area', 'name'),
                                       'lat' : extract_data(vacancy, 'address', 'lat'),
                                       'lng' : extract_data(vacancy, 'address', 'lgt'),
                                       'company_name' : extract_data(vacancy, 'employer', 'name' ),
                                       'salary_from' : extract_data(vacancy,'salary', 'from'),
                                       'salary_to' : extract_data(vacancy,'salary', 'to'),
                                       'experience' : extract_data(vacancy,'experience', 'name' ),
                                       'description' : vacancy.get('description'),
                                       'skills' : [skill.get('name') for skill in vacancy['key_skills']]}

In [ ]:
df_test

,vacancy_name,city,lat,lng,company_name,salary_from,salary_to,experience,description,skills
0,Junior data analyst,Москва,NaN,None,Lesta Games,None,None,От 1 года до 3 лет,<p><strong>Lesta Games </strong>ведёт разработ...,[Анализ данных]
1,Product\Data Analyst,Москва,NaN,None,Aviasales.ru,None,None,От 3 до 6 лет,<p><strong>Команда General Analytics занимаетс...,"[SQL, Python]"
2,Data Analyst,Новосибирск,NaN,None,Финтех Айкью,None,None,От 1 года до 3 лет,"<p>Мы ищем Data Analyst, которому будет интере...","[Python, SQL, Git, Аналитическое мышление, Ана..."
3,Аналитик данных (Big Data),Москва,NaN,None,"Лента, федеральная розничная сеть",None,None,От 1 года до 3 лет,<p>В центральном офисе федеральной розничной с...,"[SQL, Power BI, DAX, PowerBI]"
4,Data Analyst,Москва,NaN,None,билайн,None,None,От 1 года до 3 лет,<p>Сегодня <strong>билайн </strong>— компания ...,"[SQL, Python, Анализ данных, ETL, Data Analysi..."
...,...,...,...,...,...,...,...,...,...,...
7995,Senior QA Automation,Москва,55.75476,None,"ЦУМ, ТД",NaN,NaN,От 3 до 6 лет,<p>Торговый Дом ЦУМ - лидер luxury-сегмента эл...,[]
7996,Java Core разработчик (middle/senior),Екатеринбург,56.849531,None,"ДИКСИ, группа компаний",150000,300000,От 3 до 6 лет,<strong>Функции:</strong> <ul> <li>Доработка с...,"[Java, Git, ORACLE, SQL]"
7997,Senior Product Analyst,Москва,NaN,None,МФК ВЭББАНКИР,None,300000,От 3 до 6 лет,<p><strong>Компания Вэббанкир - финтех с 11 ле...,"[Python, Анализ данных, Data Analysis, Навыки ..."
7998,Middle/Middle+ backend developer (Node.js),Москва,NaN,None,Смартекс,200000,300000,От 3 до 6 лет,"<p>Привет!</p> <p>Мы <strong>Smartex</strong>,...","[JavaScript, Node.js]"


In [ ]:
vacancy_name = []
city = []
salary_from = []
salary_to = []
company_name = []
experience = []
description = []
skills = []
lat = []
lng = []

In [ ]:
for vacancy in vacancies_info:
  vacancy_name.append(vacancy["name"])
  city.append(vacancy['area']['name'])
  company_name.append(vacancy['employer']['name'])
  experience.append(vacancy['experience']['name'])
  description.append(vacancy['description'])
  skills.append([skill['name'] for skill in vacancy['key_skills']])

  if vacancy['address'] is not None:
    lat.append(vacancy['address']['lat'])
    lng.append(vacancy['address']['lng'])
  else:
    lat.append(None)
    lng.append(None)


  if vacancy['salary'] is not None:
    salary_from.append(vacancy['salary']['from'])
    salary_to.append(vacancy['salary']['to'])

  else:
    salary_from.append(None)
    salary_to.append(None)


In [ ]:
df = pd.DataFrame({'name': vacancy_name,
                  'city': city,
                  'lat': lat,
                  'lng': lng,
                  'company_name': company_name,
                  'experience': experience,
                  'skills' : skills,
                  'salary_from': salary_from,
                  'salary_to': salary_to,
                  'description': description})

In [ ]:
df

,name,city,lat,lng,company_name,experience,skills,salary_from,salary_to,description
0,Junior data analyst,Москва,NaN,NaN,Lesta Games,От 1 года до 3 лет,[Анализ данных],NaN,NaN,<p><strong>Lesta Games </strong>ведёт разработ...
1,Product\Data Analyst,Москва,NaN,NaN,Aviasales.ru,От 3 до 6 лет,"[SQL, Python]",NaN,NaN,<p><strong>Команда General Analytics занимаетс...
2,Data Analyst,Новосибирск,NaN,NaN,Финтех Айкью,От 1 года до 3 лет,"[Python, SQL, Git, Аналитическое мышление, Ана...",NaN,NaN,"<p>Мы ищем Data Analyst, которому будет интере..."
3,Аналитик данных (Big Data),Москва,NaN,NaN,"Лента, федеральная розничная сеть",От 1 года до 3 лет,"[SQL, Power BI, DAX, PowerBI]",NaN,NaN,<p>В центральном офисе федеральной розничной с...
4,Data Analyst,Москва,NaN,NaN,билайн,От 1 года до 3 лет,"[SQL, Python, Анализ данных, ETL, Data Analysi...",NaN,NaN,<p>Сегодня <strong>билайн </strong>— компания ...
...,...,...,...,...,...,...,...,...,...,...
1995,Junior Project Manager (Удалённо),Самара,NaN,NaN,Roose Agency,От 1 года до 3 лет,"[Ведение переговоров, Ориентация на результат,...",30000.0,NaN,"<p>Привет, соискатель! У тебя появилась уникал..."
1996,Junior Project Manager (Удалённо),Казань,NaN,NaN,Roose Agency,От 1 года до 3 лет,"[Ведение переговоров, Ориентация на результат,...",30000.0,NaN,"<p>Привет, соискатель! У тебя появилась уникал..."
1997,Junior Project Manager (Удалённо),Ростов-на-Дону,NaN,NaN,Roose Agency,От 1 года до 3 лет,"[Ведение переговоров, Ориентация на результат,...",30000.0,NaN,"<p>Привет, соискатель! У тебя появилась уникал..."
1998,Java Middle Developer,Москва,55.802731,37.583784,МигКредит,От 1 года до 3 лет,"[Java, SQL, Docker, Spring, Microservices, Jav...",NaN,NaN,<strong>Описание проекта:</strong><br />Наша б...


In [ ]:
def calculate_average(row):
    if not np.isnan(row['salary_from']) and not np.isnan(row['salary_to']):
        return (row['salary_from'] + row['salary_to']) / 2
    elif not np.isnan(row['salary_from']):
        return row['salary_from']
    elif not np.isnan(row['salary_to']):
        return row['salary_to']
    else:
        return np.nan

In [ ]:
df['average_salary'] = df.apply(calculate_average, axis=1)

In [ ]:
df['skills'] = df['skills'].apply(lambda skills: ', '.join(skills))

In [ ]:
df['skills'] = df['skills'].replace('', np.nan)

In [ ]:
def clean_html(text):
    text = text.lower()
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

In [ ]:
def remove_punctuation(text):
  signs = string.punctuation + '—»«–…'
  return "".join([sign for sign in text if sign not in signs])

In [ ]:
def remove_punctuation_new(text):
    translator = str.maketrans('', '', string.punctuation + '—»«–…')
    return text.translate(translator)

In [ ]:
df['description'] = df['description'].apply(clean_html)


In [ ]:
df['description'] = df['description'].apply(remove_punctuation_new)

In [ ]:
df.to_csv('data_2.csv')

In [ ]:
df

,name,city,lat,lng,company_name,experience,skills,salary_from,salary_to,description,average_salary
0,Junior data analyst,Москва,NaN,NaN,Lesta Games,От 1 года до 3 лет,Анализ данных,NaN,NaN,lesta games ведёт разработку игровых проектов ...,NaN
1,Product\Data Analyst,Москва,NaN,NaN,Aviasales.ru,От 3 до 6 лет,"SQL, Python",NaN,NaN,команда general analytics занимается организац...,NaN
2,Data Analyst,Новосибирск,NaN,NaN,Финтех Айкью,От 1 года до 3 лет,"Python, SQL, Git, Аналитическое мышление, Анал...",NaN,NaN,мы ищем data analyst которому будет интересно ...,NaN
3,Аналитик данных (Big Data),Москва,NaN,NaN,"Лента, федеральная розничная сеть",От 1 года до 3 лет,"SQL, Power BI, DAX, PowerBI",NaN,NaN,в центральном офисе федеральной розничной сети...,NaN
4,Data Analyst,Москва,NaN,NaN,билайн,От 1 года до 3 лет,"SQL, Python, Анализ данных, ETL, Data Analysis...",NaN,NaN,сегодня билайн компания изменений чутко реаги...,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1995,Junior Project Manager (Удалённо),Самара,NaN,NaN,Roose Agency,От 1 года до 3 лет,"Ведение переговоров, Ориентация на результат, ...",30000.0,NaN,привет соискатель у тебя появилась уникальная ...,30000.0
1996,Junior Project Manager (Удалённо),Казань,NaN,NaN,Roose Agency,От 1 года до 3 лет,"Ведение переговоров, Ориентация на результат, ...",30000.0,NaN,привет соискатель у тебя появилась уникальная ...,30000.0
1997,Junior Project Manager (Удалённо),Ростов-на-Дону,NaN,NaN,Roose Agency,От 1 года до 3 лет,"Ведение переговоров, Ориентация на результат, ...",30000.0,NaN,привет соискатель у тебя появилась уникальная ...,30000.0
1998,Java Middle Developer,Москва,55.802731,37.583784,МигКредит,От 1 года до 3 лет,"Java, SQL, Docker, Spring, Microservices, Java EE",NaN,NaN,описание проектанаша большая команда выполняет...,NaN


# Анализ и графики

In [ ]:
df = pd.read_csv('/content/data_2.csv')

In [ ]:
df.isna().sum()

Unnamed: 0           0
name                 0
city                 0
lat               1184
lng               1184
company_name         0
experience           0
skills             636
salary_from       1696
salary_to         1759
description          0
average_salary    1611
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      2000 non-null   int64  
 1   name            2000 non-null   object 
 2   city            2000 non-null   object 
 3   lat             816 non-null    float64
 4   lng             816 non-null    float64
 5   company_name    2000 non-null   object 
 6   experience      2000 non-null   object 
 7   skills          1364 non-null   object 
 8   salary_from     304 non-null    float64
 9   salary_to       241 non-null    float64
 10  description     2000 non-null   object 
 11  average_salary  389 non-null    float64
dtypes: float64(5), int64(1), object(6)
memory usage: 187.6+ KB


In [ ]:
def plot_top_skills(df, column_name, top_n=10):
    df = df.dropna(subset=[column_name])
    # percs = df.skills.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
    skills_list = df[column_name].str.split(', ').explode()
    top_skills = skills_list.value_counts().head(top_n)

    top_skills_percent = []
    total_count = len(skills_list)
    for skill, count in top_skills.items():
        percentage = (count / total_count) * 100
        top_skills_percent.append([skill, percentage])


    top_skills_df = pd.DataFrame(top_skills_percent, columns=['Skill', 'Percentage'])
    fig = px.pie(top_skills_df, names='Skill', values='Percentage', hole=0.3, title='Top Skills Distribution')
    fig.update_traces(textinfo='percent+label')
    fig.show()
    return top_skills





In [ ]:
plot_top_skills(df, 'skills', top_n=10)

SQL                       652
Python                    457
Анализ данных             207
ETL                       163
Аналитическое мышление    160
PostgreSQL                150
Big Data                  136
Data Analysis             125
DWH                       109
Power BI                  108
Name: skills, dtype: int64

In [ ]:
top_cities = (df.groupby('city')
              .agg(count=('city', 'count'), average_salary=('average_salary', 'mean'))
              .dropna(subset=['average_salary'])
              .sort_values('count', ascending=False)
              .head(10))


top_cities.columns = ['count_of_vacancies', 'average_salary']

top_cities['average_salary'] = top_cities['average_salary'].round(0).astype('int')


In [ ]:
top_cities

,count_of_vacancies,average_salary
city,,
Москва,1485,177679
Санкт-Петербург,226,134346
Екатеринбург,40,139818
Новосибирск,38,158500
Нижний Новгород,27,161200
Казань,25,222195
Краснодар,16,125000
Ростов-на-Дону,13,69000
Воронеж,10,179380


In [ ]:
fig = px.bar(top_cities,
             x=top_cities.index,
             y='count_of_vacancies',
             color='average_salary',
             labels={'count':'count of vacancies'},
             log_y = True )

fig.show()

In [ ]:
fig = go.Figure(go.Scattermapbox(lat=df['lat'],
                                 lon=df['lng'],
                                 text=df['city'],
                                 mode = 'markers',
                                 marker = dict(
                                 color = df['average_salary'],
                                 cmin = 0,
                                 cmax = df['average_salary'].max(),
                                 colorbar_title="salary")))

capital = df[df['city']=='Москва']
map_center = go.layout.mapbox.Center(lat=capital['lat'][0], lon=capital['lng'][0])
fig.update_layout(mapbox_style="open-street-map",
                  title = 'Distribution of vacancies on the map',
                  mapbox=dict(center=map_center, zoom=3))


fig.show()


In [ ]:
experience = df.groupby('experience').agg({"city":'count', 'average_salary': np.nanmean})
experience['average_salary'] = experience['average_salary'].round(0).astype('int')
experience.columns = ['count_of_vacancies', 'average_salary']
experience

,count_of_vacancies,average_salary
experience,,
Более 6 лет,112,226750
Нет опыта,69,79432
От 1 года до 3 лет,863,135863
От 3 до 6 лет,956,204792


In [ ]:

fig = px.bar(experience.reset_index(), x='experience', y='count_of_vacancies',
             color='average_salary',
             height=500,
             title = 'Distribution of vacancies from experience',
             labels={'count_of_vacancies':'count of vacancies'})
fig.show()

In [ ]:
class Cat():
    def __init__(self, age, weight):
        self.age = age
        self.weight = weight

    def serialize(self):
        d = dict()
        d['age'] = self.age
        d['weight'] = self.weight

    def deserialize(d):
        return Cat(
            age=d.get('age', 1), # типо если нет инфы коту годm
            weight=d.get('weight', None) # а этот параметр мы не хотим рандомно из головы
        )